In [1]:
#!/usr/bin/env python
# coding: utf-8

import pandas as pd
import numpy as np
import re
from shapely.geometry import Point
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from collections import Counter
from sklearn.ensemble import IsolationForest, HistGradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
import joblib
from sklearn.metrics import classification_report


DATA_DIR = '/cluster/home/nteutschm/eqdetection/data/'
RANDOM_STATE = 86
MODEL_TYPE = 'RandomForest' # IsolationForest HistGradientBoosting RandomForest
MODEL_PATH = f'/cluster/scratch/nteutschm/eqdetection/models/{MODEL_TYPE}.pkl'
SAVE_PATH = f'/cluster/scratch/nteutschm/eqdetection/predictions/{MODEL_TYPE}.csv'
STORE_FEATURES = f'/cluster/scratch/nteutschm/eqdetection/features/{MODEL_TYPE}'

# Loading the Data

In [2]:
def get_offsets(header_lines):
    """
    Extracts offset and postseismic decay information from the header lines of a GNSS file.

    The function captures both coseismic and non-coseismic offsets, along with postseismic decays, for 
    north (N), east (E), and up (U) components. It parses lines starting with '#' and collects the relevant 
    values into a structured dictionary categorized by the component.

    Parameters:
    - header_lines (list of str): Lines from the file that contain metadata and comments starting with '#'.

    Returns:
    - components (dict): A dictionary with keys 'n', 'e', and 'u' representing the north, east, and up components.
      Each component holds a dictionary with:
        - 'offsets': A list of dictionaries containing offset information (value, error, date, coseismic flag).
        - 'ps_decays': A list of dictionaries containing postseismic decay information (value, error, tau, date, type).
    """
    
    # Capture important information from the header
    offset_pattern = re.compile(r"#\s*(\*?)\s*offset\s+\d+:?\s+([-\d.]+)\s+\+/\-\s+([-\d.]+)\s+mm.*?\((\d{4}-\d{2}-\d{2}).*?\)")
    ps_decay_pattern = re.compile(r'#!?\s*ps decay\s+\d:\s*(-?\d+\.\d+)\s+\+/-\s+(\d+\.\d+)\s+mm\s+\((\d{4}-\d{2}-\d{2})\s+\[(\d{4}\.\d+)\]\);\s*tau:\s*(\d+)\s+days')
    component_pattern = re.compile(r"#\s+([neu])\s+component")

    components = {'n': {'offsets': [], 'ps_decays': []}, 'e': {'offsets': [], 'ps_decays': []}, 'u': {'offsets': [], 'ps_decays': []}}
    current_component = None

    for line in header_lines:
        comp_match = component_pattern.match(line)
        if comp_match:
            current_component = comp_match.group(1)
            continue

        # Check for offset
        offset_match = offset_pattern.match(line)
        if offset_match and current_component:
            coseismic = bool(offset_match.group(1))  # True if * present, meaning coseismic
            offset_value = float(offset_match.group(2))
            offset_error = float(offset_match.group(3))
            offset_date = offset_match.group(4)
            components[current_component]['offsets'].append({
                'value': offset_value,
                'error': offset_error,
                'date': offset_date,
                'coseismic': coseismic
            })

        # Check for postseismic decay
        ps_decay_match = ps_decay_pattern.match(line)
        if ps_decay_match and current_component:
            decay_value = float(ps_decay_match.group(1))
            decay_error = float(ps_decay_match.group(2))
            decay_date = ps_decay_match.group(3)
            tau = int(ps_decay_match.group(5))
            # Determine decay type based on the presence of '!'
            decay_type = 'logarithmic' if '!' in line else 'exponential'
            components[current_component]['ps_decays'].append({
                'value': decay_value,
                'error': decay_error,
                'tau': tau,
                'date': decay_date,
                'type': decay_type
            })

    return components

def read_file(filename):
    """
    Reads a GNSS file, extracting both header and data information into a pandas DataFrame.

    The function processes the header to extract metadata (e.g., station coordinates, height, offsets, decays) 
    and processes the data section to extract time-series GNSS measurements. It combines these into a DataFrame 
    with attributes containing additional metadata.

    Parameters:
    - filename (str): The path to the file containing GNSS data.

    Returns:
    - data (pandas.DataFrame): A DataFrame containing the time-series GNSS data (N, E, U components, sigmas, correlations),
      indexed by date. The DataFrame has additional attributes storing station geometry (latitude, longitude), height, 
      and offset/decay information.
    """
    
    with open(DATA_DIR+filename, 'r') as file:
        lines = file.readlines()

    header_lines = [line for line in lines if line.startswith('#')]
    if header_lines:
        column_names = re.split(r'\s{2,}', header_lines[-1].lstrip('#').strip())
    else:
        column_names = []
        
    data_lines = []
    for line in lines:
        if not line.startswith('#'):
            parts = line.strip().split()
            # Check if the number of parts matches the expected number of columns
            if len(parts) < len(column_names):
                # Add None for missing values
                parts.extend([None] * (len(column_names) - len(parts)))
            data_lines.append(parts)

    data = pd.DataFrame(data_lines)
    data.columns = column_names
    
    # Extracts latitude, longitude and height
    pattern = r'Latitude\(DD\)\s*:\s*(-?\d+\.\d+)|East Longitude\(DD\)\s*:\s*(-?\d+\.\d+)|Height\s*\(M\)\s*:\s*(-?\d+\.\d+)'
    #referece_pattern = r'Reference_X\s*:\s*(-?\d+\.\d+)|Reference_Y\s*:\s*(-?\d+\.\d+)|Reference_Z\s*:\s*(-?\d+\.\d+)'
    matches = re.findall(pattern, ' '.join(header_lines))
    geom = Point(float(matches[1][1]), float(matches[0][0]))
    
    offsets = get_offsets(header_lines)

    data['Date'] = pd.to_datetime(data['Yr'].astype(str) + data['DayOfYr'].astype(str), format='%Y%j')
    data.set_index('Date', inplace=True)
    data.drop(['Dec Yr', 'Yr', 'DayOfYr', 'Chi-Squared'], axis=1, inplace=True)
    cols = ['N', 'E', 'U', 'N sig', 'E sig', 'U sig', 'CorrNE', 'CorrNU', 'CorrEU']
    data[cols] = data[cols].astype(float)
    
    data.name = filename.replace("RawTrend.neu", "")
    data.attrs['geometry'] = geom
    data.attrs['height'] = float(matches[2][2])
    data.attrs['offsets'] = offsets
    
    return data

# Cleaning Data

In [3]:
def add_missing_dates(df):
    df.index = pd.to_datetime(df.index)
    full_date_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='D')
    df_full = df.reindex(full_date_range)
    df_full.name = df.name
    return df_full

In [4]:
def clean_dataframes(dfs, missing_value_threshold=None):
    """
    Cleans the dataframes by:
    1. Removing dataframes without any coseismic offsets in any of the 3 components (n, e, u).
    2. Removing non-coseismic offsets from all components.
    3. Optionally removing dataframes with excessive missing values in all 3 components.

    Parameters:
    dfs (list): List of dataframes with GNSS data.
    missing_value_threshold (float, optional): Percentage (0 to 1) of allowed missing values.
                                               If exceeded, the dataframe is removed.

    Returns:
    list: Cleaned list of dataframes.
    """

    cleaned_dfs = []
    components = ['N', 'E', 'U']
    components_offsets = ['n', 'e', 'u']

    for org_df in dfs:
        
        has_coseismic = False
        df = add_missing_dates(org_df)

        for comp in components_offsets:
            filtered_offsets = []
            for offset in df.attrs['offsets'][comp]['offsets']:
                if offset['coseismic']:
                    has_coseismic = True
                    filtered_offsets.append(offset)
            # Update offsets to retain only coseismic
            df.attrs['offsets'][comp]['offsets'] = filtered_offsets

        # Skip dataframe if no coseismic offsets in any component
        if not has_coseismic:
            continue

        # Check missing values for all components combined, if threshold is provided
        if missing_value_threshold is not None:
            total_values = sum(df[comp].size for comp in components)
            missing_values = sum(df[comp].isna().sum() for comp in components)

            missing_percentage = missing_values / total_values
            if missing_percentage > missing_value_threshold:
                continue  # Skip the dataframe if missing values exceed the threshold

        # Add the cleaned dataframe to the list
        cleaned_dfs.append(df)

    return cleaned_dfs

# Machine Learning

In [5]:
def extract_features(dfs, interpolate=True, chunk_size=21):
    """
    Extracts relevant features from a list of dataframes, including displacement values, 
    errors, offsets, decay information, station locations, and heights.

    Parameters:
    dfs (list): List of dataframes with GNSS data.
    interpolate (bool): Whether to interpolate missing values or retain `None`.
    chunk_size (int): Number of consecutive days to combine into one sample (row).

    Returns:
    Tuple (DataFrame, list): Combined dataframe with extracted features, and the target vector.
    """
    feature_matrix = []
    target_vector = []
    components_offsets = ['n', 'e', 'u']  # North, East, Up
    
    #columns to include in creating the chunks
    #cols = ['N', 'E', 'U', 'N sig', 'E sig', 'U sig', 'CorrNE', 'CorrNU', 'CorrEU']
    cols = ['N', 'E', 'U']

    for df in dfs:
        # Extract basic features (displacement, errors, correlations)
        features = df[['N', 'E', 'U', 'N sig', 'E sig', 'U sig', 'CorrNE', 'CorrNU', 'CorrEU']].copy()

        if interpolate:
            features.interpolate(method='time', inplace=True)

        # Get station location and height information
        location = df.attrs.get('geometry')
        if isinstance(location, Point):
            latitude, longitude = location.y, location.x
        else:
            latitude, longitude = None, None
        height = df.attrs.get('height', None)

        # Extract offsets and decay information for each component
        for comp in components_offsets:
            series_names = ['offset_value', 'offset_error', 'decay_value', 'decay_error', 'decay_tau', 'decay_type']
            series_dict = {name: pd.Series(0.0 if interpolate else None, dtype='float64', index=df.index) for name in series_names}
            series_dict['decay_type'] = pd.Series(0.0, dtype='int64', index=df.index)  # decay_type should be int

            for offset in df.attrs['offsets'][comp]['offsets']:
                series_dict['offset_value'].loc[offset['date']] = offset['value']
                series_dict['offset_error'].loc[offset['date']] = offset['error']

            for decay in df.attrs['offsets'][comp]['ps_decays']:
                series_dict['decay_value'].loc[decay['date']] = decay['value']
                series_dict['decay_error'].loc[decay['date']] = decay['error']
                series_dict['decay_tau'].loc[decay['date']] = decay['tau']
                series_dict['decay_type'].loc[decay['date']] = 1 if decay['type'] == 'logarithmic' else 2

            # Add series to features
            for name, series in series_dict.items():
                features[f'{comp}_{name}'] = series

        # Add station metadata (location and height)
        features['latitude'] = latitude
        features['longitude'] = longitude
        features['height'] = height

        # Create the feature matrix with chunking
        for i in range(len(features) - chunk_size + 1):
            # Create a chunk of size `chunk_size` for each feature (ignore offsets and decays as it takes too much memory (more than 80GB))
            #feature_row = np.hstack([features[col].values[i:i + chunk_size] for col in features.columns])
            feature_row = np.hstack([features[col].values[i:i + chunk_size] for col in cols])

            # Add the row to the feature matrix
            feature_matrix.append(feature_row)

            # Determine the target value for this chunk: index of earthquake or 0 if none
            offset_values_chunk = features[['n_offset_value', 'e_offset_value', 'u_offset_value']].iloc[i:i + chunk_size]
            if (offset_values_chunk != 0).any().any():
                target_vector.append(1)  # earthquake/discontinuity in this chunk
            else:
                target_vector.append(0)  # no discontinuity

    feature_matrix = np.array(feature_matrix)
    return pd.DataFrame(feature_matrix), target_vector

In [6]:
def save_predictions(test_predictions):
    test_predictions.to_csv(SAVE_PATH, index=True)
    
def compute_weights(train_labels):
    classes = np.unique(train_labels)
    class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=train_labels)
    return {c: w for c, w in zip(classes, class_weights)}

In [7]:
def prepare_data(X, y, test_size=0.3, random_state=42):
    """
    Prepares training and testing data by splitting the feature matrix and target vector.

    Parameters:
    X (DataFrame): The feature matrix (chunked GNSS data).
    y (list or Series): The target labels (0 for no offset, 1 for offset).
    test_size (float): Proportion of the dataset to include in the test split.
    random_state (int): Random seed for reproducibility.

    Returns:
    X_train (DataFrame): Training set feature matrix.
    X_test (DataFrame): Test set feature matrix.
    y_train (Series): Training set target vector.
    y_test (Series): Test set target vector.
    class_weights (dict): Weights for handling imbalanced classes.
    """
    # Split into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    # Convert target lists to Pandas Series if necessary
    if isinstance(y_train, list):
        y_train = pd.Series(y_train)
    if isinstance(y_test, list):
        y_test = pd.Series(y_test)

    # Compute class weights to handle imbalanced dataset
    class_weights = compute_weights(y_train)
    
    return X_train, X_test, y_train, y_test, class_weights

In [8]:
def optimize_random_forest(X_train, y_train, weights):
    param_grid = {
        'n_estimators': [100, 500],
        'max_depth': [10, 30],
        'class_weight': [weights], 
        'random_state': [RANDOM_STATE]
    }
    rf = RandomForestClassifier()
    stratified_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
    grid_search = GridSearchCV(rf, param_grid, cv=stratified_cv, scoring='f1_weighted', verbose=3, n_jobs=-1, pre_dispatch='2*n_jobs')
    grid_search.fit(X_train, y_train)
    
    print("Best RandomForest parameters found: ", grid_search.best_params_)
    
    return grid_search.best_estimator_

def optimize_isolation_forest(X_train, y_train):
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_samples': [0.8, 1.0],
        'contamination': [0.001, 0.002, 0.005],
        'random_state': [RANDOM_STATE]
    }
    iso_forest = IsolationForest()
    stratified_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
    grid_search = GridSearchCV(iso_forest, param_grid, cv=stratified_cv, scoring='f1_weighted', verbose=1, n_jobs=-1, pre_dispatch='2*n_jobs')
    # Have to give the labels as input even though IsolationForest uses no labels and the input is optional as otherwise there is an error. No idea why. 
    grid_search.fit(X_train, y_train)
    
    print("Best IsolationForest parameters found: ", grid_search.best_params_)
    
    return grid_search.best_estimator_

def optimize_hist_gradient_boosting(X_train, y_train, weights):
    param_grid = {
        'learning_rate': [0.01, 0.1, 0.2],
        'max_iter': [100, 200, 300],
        'max_depth': [5, 10, 15],
        'class_weight': [weights],
        'random_state': [RANDOM_STATE]
    }
        
    hgb = HistGradientBoostingClassifier()
    stratified_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
    grid_search = GridSearchCV(
        estimator=hgb,
        param_grid=param_grid,
        cv=stratified_cv,
        scoring='f1_weighted',
        verbose=1,
        n_jobs=-1,
        pre_dispatch='2*n_jobs'
    )
    grid_search.fit(X_train, y_train)
    
    print("Best HistGradientBoosting parameters found: ", grid_search.best_params_)
    
    return grid_search.best_estimator_

In [9]:
def train_model(X, y, model_type):
    """
    Trains a model based on the specified type using both North, East, and Up component data.

    Parameters:
    X (DataFrame): Feature set, including target values.
    model_type (str): The type of model to train ('IsolationForest' or 'HistGradientBoosting').

    Returns:
    model: Trained model.
    test_predictions: Predictions on the test set.
    report: Classification report for the test set.
    """
    
    X_train, X_test, train_labels, test_labels, weights = prepare_data(X, y)
    
    # Imbalance test: No offsets: 2133084, offsets: 525

    # Set offset/decay columns based on model type
    if model_type == 'IsolationForest':
        # For IsolationForest, ignore decays and offset information (unsupervised)
        #columns = [col for col in X_test.columns if 'offset' in col or 'decay' in col]
        #X_test.drop(columns, axis=1, inplace=True)
        #X_train.drop(columns, axis=1, inplace=True)
        model = optimize_isolation_forest(X_train, train_labels)
        test_predictions = model.predict(X_test)
        
    elif model_type == 'RandomForest':
        # For RandomForest, ignore decays and offset information
        #columns = [col for col in X_test.columns if 'offset' in col or 'decay' in col]
        #X_test.drop(columns, axis=1, inplace=True)
        #X_train.drop(columns, axis=1, inplace=True)
        model = optimize_random_forest(X_train, train_labels, weights)
        test_predictions = model.predict(X_test)
        
    
    elif model_type == 'HistGradientBoosting':
        # For HistGradientBoosting, set offset and decay columns to None in test data (supervised using binary training labels)
        #X_test[[col for col in X_test.columns if 'offset' in col or 'decay' in col]] = None
        model = optimize_hist_gradient_boosting(X_train, train_labels, weights)
        test_predictions = model.predict(X_test)
        
    else:
        raise ValueError('Used Model Type not implement. Please control spelling!')
    
    test_predictions = pd.Series(test_predictions, index=X_test.index)
    
    joblib.dump(model, MODEL_PATH)
    save_predictions(test_predictions)
    
    if model_type == 'IsolationForest':
        test_predictions = test_predictions.map({-1: 1, 1: 0})
    report = classification_report(test_labels, test_predictions, target_names=['No Coseismic Event', 'Coseismic Event'])
    
    print(f"Class weights used during training:\nNo Coseismic Event: {weights[0]} \nCoseismic Event: {weights[1]}")
    
    return model, test_predictions, report

In [10]:
def main():
    dfs = []
    dir = Path(DATA_DIR)
    for file_path in dir.iterdir():
        if file_path.is_file():
            dfs.append(read_file(file_path.name))

    cleaned_dfs = clean_dataframes(dfs, missing_value_threshold=0.05)
    
    # HistGradientBoosting designed to deal with None data -> No interpolation needed
    interpolate = False if MODEL_TYPE == 'HistGradientBoosting' else True
    X, y = extract_features(cleaned_dfs, interpolate=interpolate)
    
    X.to_csv(f'{STORE_FEATURES}_features.csv', index=True)
    pd.Series(y).to_csv(f'{STORE_FEATURES}_target.csv', index=False, header=False)
    
    model, test_predictions, report = train_model(X, y, model_type=MODEL_TYPE)
    print(f'Report for model: {MODEL_TYPE} \n {report}')

In [11]:
if __name__=='__main__':
    main()

"if __name__=='__main__':\n    main()"